In [1]:
seed = 123
import time
import pandas as pd
import numpy as np
import os
import haiku as hk
import jax
import jax.numpy as jnp
import nucleotide_transformer
from nucleotide_transformer.mypretrained import get_pretrained_model
import time

import sys
sys.path.insert(0, '..')
from config import *

In [2]:
set_data = 'training'
assert set_data in ['training', 'val', 'test']
meta = pd.read_csv(os.path.join(metadata_dir, f'{set_data}.csv'))

In [30]:
# excluded = pd.read_csv(os.path.join(metadata_dir, 'excluded_training.txt'), header = None)
# done = pd.read_csv(os.path.join(metadata_dir, 'done_training.txt'), header = None)
# set1 = meta[(meta.id_sample.isin(excluded[0].values))].cdna1.apply(lambda x: set(x))
# set2 = meta[(meta.id_sample.isin(excluded[0].values))].cdna2.apply(lambda x: set(x))
# meta[~meta.id_sample.isin(set(done[0]))].to_csv(os.path.join(metadata_dir, 'training_remain.csv'), index = False)

### NT Embeddings

In [ ]:
random_key = jax.random.PRNGKey(0)

In [56]:
def calculate_grouped_mean_embeddings(outs, layer, tokens, tokenizer, k):
    # Get the embeddings for the specified layer
    embeddings = outs[f"embeddings_{layer}"]
    
    # Remove the CLS token and paddings
    embeddings = embeddings[:, 1:, :]
    padding_mask = jnp.expand_dims(tokens[:, 1:] != tokenizer.pad_token_id, axis=-1)
    masked_embeddings = embeddings * padding_mask
    
    # Calculate the number of groups
    batch_size = masked_embeddings.shape[0]
    seq_length = masked_embeddings.shape[1]
    num_groups = seq_length // k
    
    # Reshape the embeddings to form groups
    grouped_embeddings = jnp.reshape(masked_embeddings[:, :num_groups*k, :], (batch_size, num_groups, k, -1))
    grouped_padding_mask = jnp.reshape(padding_mask[:, :num_groups*k, :], (batch_size, num_groups, k, -1))
    
    # Calculate the mean embeddings for each group
    group_sum_embeddings = jnp.sum(grouped_embeddings, axis=2)
    sequence_count_in_groups = grouped_padding_mask.sum(axis=2)
    sequence_count_in_groups = jnp.where(sequence_count_in_groups == 0, 1, sequence_count_in_groups)  # to avoid division by zero
    group_mean_embeddings = group_sum_embeddings / sequence_count_in_groups
    
    if k == 999:
        group_mean_embeddings = group_mean_embeddings.squeeze()
    
    return group_mean_embeddings

def infer(sequences, forward_fn, tokenizer, parameters, random_key):
    tokens_ids = [b[1] for b in tokenizer.batch_tokenize(sequences)]
    tokens_str = [b[0] for b in tokenizer.batch_tokenize(sequences)]
    tokens = jnp.asarray(tokens_ids, dtype=jnp.int32)
    # Infer
    outs = forward_fn.apply(parameters, random_key, tokens)
    return outs, tokens

def save_data_to_folder(data, labels, ids, folder_path):
    if not os.path.exists(folder_path):
        os.makedirs(folder_path)
    for i in range(len(data)):
        sample = data[i]
        label = labels[i]
        id_sample = ids[i]
        if label == 0:
            class_folder = os.path.join(folder_path, 'class_0')
        else:
            class_folder = os.path.join(folder_path, 'class_1')
        if not os.path.exists(class_folder):
            os.makedirs(class_folder)
        np.save(os.path.join(class_folder, f'{id_sample}.npy'), sample)

In [ ]:
embeddings_layers_to_save = (22, 30)

model_name = '2B5_multi_species'

# Get pretrained model
parameters, forward_fn, tokenizer, config = get_pretrained_model(
    model_name=model_name,
    mixed_precision=False,
    embeddings_layers_to_save=embeddings_layers_to_save,
    attention_maps_to_save=(),
    max_positions=1000,
    chkpt_dir = os.path.join(nt_dir, 'checkpoints')
)
forward_fn = hk.transform(forward_fn)
print('Model loaded')

In [8]:
batch_size = 13
k = 999
n_batch = int(meta.shape[0]/batch_size)
slices = np.linspace(0, meta.shape[0], n_batch, dtype = np.int64)

k_dir = os.path.join(embedding_dir, str(k))
if not os.path.exists(k_dir):
    os.makedirs(k_dir)

In [9]:
start_time = time.time()
for i in range(len(slices)-1):
    
    meta_slice = meta[slices[i]:slices[i+1]]
    labels = list(meta[slices[i]:slices[i+1]].interacting.values.astype(int))
    ids = list(meta[slices[i]:slices[i+1]].id_sample.values)
    
    try:
        sequences1 = list(meta_slice.cdna1.values)
        outs1, tokens1 = infer(sequences1, forward_fn, tokenizer, parameters, random_key)

        sequences2 = list(meta_slice.cdna2.values)
        outs2, tokens2 = infer(sequences2, forward_fn, tokenizer, parameters, random_key)

        for layer in embeddings_layers_to_save:
            layer_folder = os.path.join(k_dir, str(layer))
            if not os.path.exists(layer_folder):
                os.makedirs(layer_folder)

            mean_embeddings1 = calculate_grouped_mean_embeddings(outs1, layer, tokens1, tokenizer, k) #shape is (batch_size, 2560)
            mean_embeddings2 = calculate_grouped_mean_embeddings(outs2, layer, tokens2, tokenizer, k) #shape is (batch_size, 2560)

            #concatenate the two embeddings (check if I am doing this properly, with the rigth axis)
            embeddings = np.concatenate((mean_embeddings1, mean_embeddings2), axis=1) #shape is (2*batch_size, 5120)

            #save the embeddings
            save_data_to_folder(embeddings, labels, ids, os.path.join(layer_folder, set_data))

        del outs1
        del outs2
    except:
        with open(os.path.join(metadata_dir, f"excluded_{set_data}.txt"), 'a') as f:
            for idx in ids:
                f.write(str(idx) + '\n')

    with open(os.path.join(metadata_dir, f"done_{set_data}.txt"), 'a') as f:
        for idx in ids:
            f.write(str(idx) + '\n')
    
    if i%190 == 0:
        perc = np.round(i/len(slices) * 100, 2)
        print(f'{perc}% done in {(time.time()-start_time)/60} minutes')
    
print(f"Total time to process batch: {(time.time()-start_time)/60} minutes")

2023-04-15 14:55:00.453287: W external/xla/xla/service/hlo_rematerialization.cc:2194] Can't reduce memory use below 8.06GiB (8658026496 bytes) by rematerialization; only reduced to 12.00GiB (12888088964 bytes)
2023-04-15 14:55:17.713095: E external/xla/xla/pjrt/pjrt_stream_executor_client.cc:2432] Execution of replica 0 failed: INTERNAL: Failed to allocate 104857600 bytes for new constant


XlaRuntimeError: INTERNAL: Failed to allocate 104857600 bytes for new constant